# API key
Please enter your API key below. Keep your key confidential, to prevent abuse.

To acquire an API key, go to https://old.ppy.sh/p/api/ (this link is funky. You may need to login and then follow the link again a few times. @ ppy not me)

In [ ]:
API_KEY = input("Enter your API key in here, please:\n")

# Function Declarations

Let's keep this code as clean as possible

In [ ]:
from enum import Enum

import requests

class USER_ID(Enum):
    ID = "id"
    STRING = "string"
    
class GAME_MODE(Enum):
    STANDARD = 0


def make_api_call(method_name:str, params: dict) -> dict:
    HOST = f"https://osu.ppy.sh/api"
    STANDARD = 0
    request_params = {
        "k": API_KEY,
        "m": GAME_MODE.STANDARD,
    }
    
    request_params.update(params)
    r = requests.get(url=f"{HOST}/{method_name}", params=request_params)
    result = r.json()
    try:
        return result[0]
    except IndexError:
        return result


class User:        
    def __init__(self, user_id: str, username: str, country_rank: int, global_rank: int, **kwargs):
        self.user_id = user_id
        self.username = username
        self.country_rank = country_rank
        self.global_rank = global_rank

    @classmethod
    def get_user_info(cls, user_id: str, id_type=USER_ID.ID) -> "User":
        response = make_api_call("get_user", {"u": user_id, "type": id_type})
        return cls(
            user_id = user_id,
            username = response.get("username"),
            country_rank = int(response.get("pp_country_rank", -1)),
            global_rank = int(response.get("pp_rank", -1)),
        )
    

# Now for the actual code
...

In [ ]:
USERS = []
# this should be async, but is fine for demo purposes
with open("top100.txt", "r") as f:
    for user_id in f.readlines():
        user_id = user_id.strip() # readlines can be weird
        user = User.get_user_info(user_id)
        USERS.append(user)

In [22]:
import plotly.graph_objects as go

USERS.sort(key=lambda x: x.country_rank)

rank_ratio = [user.global_rank/user.country_rank for user in USERS]
x_axis = list(range(1, len(USERS) + 1))
fig = go.Figure(data=go.Scatter(y=rank_ratio, x=x_axis))
fig.show()